<a href="https://colab.research.google.com/github/m3llad0/water-pollution/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preprocessing

_Autor: Diego Mellado, Luis C. Juarez-Moreno_

---

## 1. Introducción

Esta sección describe en detalle el **procesamiento inicial** de las cinco fuentes de datos crudos (Excel) para preparar un conjunto limpio de CSVs que alimentarán el cálculo del Índice de Calidad del Agua (ICA) y los modelos posteriores.  
Los cinco cuerpos de agua cubiertos son:
1. Laguna de Cajititlán  
2. Río Santiago  
3. Río Zula-Lerma  
4. Laguna Zapotlán  
5. Río Verde  

### 1.1. Objetivos Específicos de la Limpieza
1. **Unificar y normalizar** los nombres de parámetros y puntos de muestreo.  
2. **Eliminar mediciones no relevantes** (solo conservar los parámetros necesarios para el ICA).  
3. **Convertir valores no numéricos** (p. ej. “<0.01”) en datos numéricos útiles.  
4. **Pivotar la tabla** para que cada registro sea una combinación única de (`Fecha`, `Punto de Muestreo`) y las columnas sean los 12 parámetros seleccionados.  
5. **Generar un CSV** limpio por cada cuerpo de agua, listo para el cálculo del ICA.

---

## 2. Montaje de Google Drive


In [ ]:
from google.colab import drive
import pandas as pd


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Definición de Parámetros y Rutas de Archivo


- `files`: nombres de los archivos Excel originales.

- `water_bodies`: nombres exactos de las hojas dentro de cada Excel.

- `water_bodies1`: versiones sin tildes/espacios para nombrar los CSV resultantes.

- `params`: lista de 12 parámetros críticos para el ICA, según CONAGUA (2014) .



In [ ]:
files = ["RioSantiago.xlsx", "RioZula-Lerma.xlsx", "LagunaCajititlan.xlsx",
         "LagunaZapotlan.xlsx", "RioVerde.xlsx"]

water_bodies = ["Río Santiago", "Río Zula-Lerma", "Laguna de Cajititlán", "Laguna Zapotlán", "Río Verde"]
water_bodies1 = ["Rio Santiago", "Rio Zula-Lerma", "Laguna de Cajititlan", "Laguna Zapotlan", "Rio Verde"]

params = [
    "Oxígeno Disuelto",
    "Demanda Bioquímica",
    "Demanda Química",
    "Potencial",
    "Conductividad",
    "Amoniacal",
    "Fósforo",
    "Suspendidos Totales",
    "Alcalinidad total",
    "Plomo",
    "Mercurio",
    "Coliformes fecales"
]

## 4. Lectura y Filtrado de las Hojas Excel
Para cada archivo (`files[i]`) se lleva a cabo:

1. Lectura de la hoja principal:

2. Limpieza de la Columna “valor”

- Qué hace:

  - Quita espacios y convierte cadenas numéricas a float.

  - Si hay un prefijo “<” (e.g., <0.01), toma el valor numérico detrás del < y divide entre 2 para aproximar “LOD/2”.
  
Técnicas de análisis químico suelen usar “LOD/2” cuando un contaminante está por debajo del umbral de detección .

Permite conservar la fila con un valor aproximado en lugar de desecharla.

In [ ]:
def clean_val(v):
    if isinstance(v, str):
        v = v.strip()
        if v.startswith("<"):
            try:
                return float(v[1:]) / 2  # Interpretar como mitad del valor de detección
            except:
                return pd.NA
        try:
            return float(v)
        except:
            return pd.NA
    return v

4. Lectura y Filtrado de las Hojas Excel
Para cada archivo (files[i]) se lleva a cabo:



3. Lectura de catálogos:

`df_params`: mapea `idParametros` → `Parametro`.
`df_points`: mapea `idPunto` → `puntoMuestreo`.

4. Filtrado de parámetros de interés:

- Se retienen solo los parámetros relacionados con el ICA (e.g., “Demanda química de oxígeno” coincide con "Demanda Química").

- Evita procesar mediciones irrelevantes y reduce el tamaño de la tabla.

5. Unión (merge) con df_main:

- `how="inner"` en la unión con `filtered_params` garantiza que solo queden filas con parámetros relevantes.

- Se añaden columnas descriptivas Parametros y Punto de Muestreo para cada medición.


6. Conversión de “Fecha” a Formato `datetime`

- Transformar la columna fecha (cadena) en un objeto datetime de pandas.

- Hacer posible ordenar cronológicamente, filtrar por rangos y calcular series de tiempo posteriormente.

errors="coerce" convierte valores inválidos en NaT. Estas filas podrán revisarse o descartar si la fecha es esencial.


In [ ]:
for i in range(len(files)):
    print(f"{i+1}. {water_bodies[i]}")
    xls = pd.ExcelFile(f"/content/drive/MyDrive/Water project/Dataset(s)/raw data/{files[i]}")

    # Leer hojas
    df_main = xls.parse(water_bodies[i])
    df_params = xls.parse("Parametros")
    df_points = xls.parse("Puntos de Muestreo")

    # Filtrar parámetros
    mask = df_params["Parametro"].str.lower().apply(
        lambda x: any(keyword.lower() in x for keyword in params)
    )
    filtered_params = df_params[mask]

    # Merge con nombre del parámetro
    df_main = df_main.merge(
        filtered_params[["idParametros", "Parametro"]],
        left_on="idParametro",
        right_on="idParametros",
        how="inner"  # <--- Important: keep only matching parameters
    )

    # Merge con nombre del punto de muestreo
    df_main = df_main.merge(
        df_points[["idPunto", "puntoMuestreo"]],
        left_on="idPuntoMuestreo",
        right_on="idPunto",
        how="left"
    )

    # Renombrar columnas
    df_main.rename(columns={
        "Parametro": "Parametros",
        "puntoMuestreo": "Punto de Muestreo"
    }, inplace=True)

    # Conservar solo columnas relevantes
    df_main = df_main[["valor", "fecha", "Parametros", "Punto de Muestreo"]]


    df_main["valor"] = df_main["valor"].apply(clean_val)


    # Rename columns for easier use
    df_main = df_main.rename(columns={
        "valor": "Valor",
        "fecha": "Fecha",
        "Parametros": "Parametros"
    })

    # Automatically parse the date column
    df_main["Fecha"] = pd.to_datetime(df_main["Fecha"], format="ISO8601", errors="coerce")

    # Optional: Check for any parsing errors
    if df_main["Fecha"].isnull().any():
        print("Some dates could not be parsed. Showing rows with issues:")
        print(df_main[df_main["Fecha"].isnull()])

    # Pivot the table
    pivot_df = df_main.pivot_table(
        index=["Fecha", "Punto de Muestreo"],
        columns="Parametros",
        values="Valor"
    )

    # Rename columns for easier use
    pivot_df = pivot_df.rename(columns={
        "Potencial de hidrógeno": "pH",
        "Demanda bioquímica de oxígeno": "DBO 5",
        "Demanda química de oxígeno": "DQO",
        "Sólidos suspendidos totales": "SST"
    })

    # Flatten columns and reset index
    pivot_df.columns.name = None
    pivot_df.reset_index(inplace=True)

    # Preview the result
    print(pivot_df.head())

    # Guardar
    output_path = f"/content/drive/MyDrive/Water project/Dataset(s)/processed/{water_bodies1[i].replace(' ', '_').lower()}.csv"
    pivot_df.to_csv(output_path, index=False)


## 5. Resultados de la Limpieza
- Se generan los siguientes archivos .csv:

1. `laguna_de_cajititlan.csv`
2. `rio_santiago.csv`
3. `rio_zula-lerma.csv`
4. `laguna_zapotlan.csv`
5. `rio_verde.csv`

- Cada uno contiene columnas exactamente:

- Fecha (datetime)
- Punto de Muestreo (texto)
Las 12 variables seleccionadas:

1. Alcalinidad total
2. Coliformes fecales
3. Conductividad
4. DBO 5
5. DQO
6. Fósforo total
7. Mercurio
8. Nitrógeno amoniacal
9. Oxígeno disuelto
10. Plomo
11. pH
12. SST


Un conjunto estructurado y limpio en formato CSV es indispensable para la fase de modelado.

Ante posibles actualizaciones de datos (p.ej. nuevas mediciones mensuales), basta con colocar el nuevo archivo Excel en Drive y rerunear únicamente esta sección de “Data Cleaning” para obtener CSVs actualizados al instante.

## Referencias

- Comisión Nacional del Agua (CONAGUA). (2014). Indicadores de la Calidad del Agua en México.
→ Base para la elección de parámetros, rangos de subíndice y definición de ICA .

- Pedregosa, F., Varoquaux, G., Gramfort, A., et al. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12, 2825–2830. (para funciones de pivot, merge y limpieza en pandas).